In [1]:
import numpy as np
import pandas as pd
import pickle

import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

pd.set_option("display.max_rows", 1000)
# pd.set_option("max_colwidth", 200)

%load_ext autoreload
%autoreload 2

In [2]:
df_videos_cleaned_v7 = pickle.load(open('../Data/df_videos_cleaned_v7.pickle', 'rb'))

### All the functions (Need to put these into a separate .py file)

In [3]:
def remove_custom_stopwords(df):
    '''
    Input: Cleaned dataframe
    Output: Dataframe with custom stopwords removed
    '''
    def final_preprocessing(cleaned_text):  
        nlp.Defaults.stop_words |= {'uh','yeah','man','um','oh','guy','maybe','bye','hey', 'sort'}
        stopwords = nlp.Defaults.stop_words
        
        preprocessed_text_12 = [(word.lower(), pos) for word, pos in cleaned_text 
                                    if word.lower() not in stopwords] 
        
        return preprocessed_text_12
    
    df['Transcript'] = df['Transcript'].apply(final_preprocessing)
            
    return df  

In [4]:
def document_term_matrix(df, vectorizer):
    '''
    Input: Cleaned dataframe (after removing custom stopwords) and type of vectorizer
    Output: Document-term matrix
    '''
    ## Take the words out of the (word, POS) tuple, vectorize, and fit-transform into a matrix
    word_list = [[word[0] for word in doc] for doc in df['Transcript']]
    vec = vectorizer(tokenizer=lambda doc:doc, lowercase=False, min_df=2, max_df=0.3)
    matrix = vec.fit_transform(word_list).toarray()
        
    return matrix, vec.get_feature_names()

In [5]:
def topic_model(matrix, model, num_topics, num_words):
    '''
    Input: Document-term matrix, type of topic model, number of topics, and number of words is each topic
    Output: a list of lists containing topic words
    '''
    ## Creates an instance of an NMF or LDA model
    if model == NMF:
        model = model(num_topics)
    elif model == LatentDirichletAllocation:
        model = model(n_components=num_topics)
        
    ## Fit_transform (matrix factorization for NMF) the doc_word matrix to get doc_topic and topic_word matrices
    doc_topic = model.fit_transform(matrix)
    topic_word = model.components_
    
    ## Retrieves the top words in each topic
    words = document_term_matrix(df_videos_cleaned_v9, CountVectorizer)[1]
    t_model = topic_word.argsort(axis=1)[:, -1:-(num_words+1):-1]
    top_topic_words = [[words[i] for i in topic] for topic in t_model]
        
    return top_topic_words, doc_topic

In [6]:
def corpus_of_adjectives(df):
    '''
    Input: Cleaned dataframe (after removing custom stopwords) 
    Output: Dataframe with only adjectives in the transcript corpus
    '''
    def adjectives(cleaned_text):
        
        preprocessed_text_adj = [(word.lower(), pos) for word, pos in cleaned_text 
                                    if pos=='ADJ'] 
        
        return preprocessed_text_adj
    
    df['Transcript'] = df['Transcript'].apply(adjectives)
            
    return df

In [7]:
def topic_assignment(df):
    '''
    Input: Cleaned dataframe (after removing custom stopwords)
    Output: Dataframe with topic and topic coefficient added
    '''
    ## Takes the highest coefficient for each video (row) in the doc_topic matrix, and puts them into a list 
    doc_topic = topic_model(X_tfidf, NMF, 12, 7)[1]
    topic_coeff = [round(np.max(coeffs),3) for coeffs in doc_topic]
    topic = list(doc_topic.argmax(axis=1))
    
    ## Map topic indices to topic names
    topic_keys = {0:'Value Investing', 1:'Valuation', 2:'Economic Moats', 3:'Passive Investing', 
                  4:'Valuation (Case Studies)', 5:'Technology Stocks', 6:'General', 7:'Value Investing', 8:'Fundamental vs. Technical Analysis', 
                  9:'Electric Vehicle Stocks', 10:'Value Investing', 11:'Dividend Investing'}
    
    topic_name = [topic_keys.get(topic_index,'') for topic_index in topic]
    
    ## Add the Topic and Topic Coefficient columns
    df['Topic'] = topic_name
    df['Topic Coefficient'] = topic_coeff
    
    return df

In [8]:
def pickle_df(df_str):
    '''
    Input: Name of a dataframe in a string format
    Output: Pickle the dataframe into the Data folder
    '''  
    with open('../Data/'+ df_str +'.pickle', 'wb') as f_video_data:
        pickle.dump(eval(df_str), f_video_data)

### Remove custom stopwords

In [9]:
df_videos_cleaned_v8 = remove_custom_stopwords(df_videos_cleaned_v7)

In [10]:
pickle_df('df_videos_cleaned_v8')

### Creating document-term matrices

In [11]:
df_videos_cleaned_v9 = pickle.load(open('../Data/df_videos_cleaned_v9.pickle', 'rb'))

#### CountVectorizer

In [12]:
X_cv = document_term_matrix(df_videos_cleaned_v9, CountVectorizer)[0]

#### TfidfVectorizer

In [13]:
X_tfidf = document_term_matrix(df_videos_cleaned_v9, TfidfVectorizer)[0]

### Topic modeling - Entire corpus

#### Non-negative matrix factorization (NMF), CountVectorizer

In [14]:
topics_nmf_cv = topic_model(X_cv, NMF, 12, 7)[0]
topics_nmf_cv

[['life', 'team', 'cheap', 'stuff', 'problem', 'answer', 'berkshire'],
 ['option', 'leap', 'decay', 'cover', 'spread', 'view', 'ge'],
 ['equal', 'divide', 'constant', 'discount', 'present', 'zero', 'minus'],
 ['graham', 'security', 'ben', 'street', 'intrinsic', 'wall', 'benjamin'],
 ['etf', 'holding', 'index', 'sector', 'expense', 'vanguard', 'individual'],
 ['bank', 'loan', 'credit', 'sector', 'percentage', 'loss', 'deposit'],
 ['unit', 'purchase', 'method', 'inventory', 'batch', 'system', 'blue'],
 ['moat', 'brand', 'customer', 'competitive', 'network', 'economic', 'mode'],
 ['tesla', 'apple', 'stuff', 'youtube', 'car', 'amazon', 'drop'],
 ['multiple', 'ebitda', 'pe', 'enterprise', 'statement', 'sheet', 'forecast'],
 ['report', 'news', 'release', 'economy', 'data', 'event', 'economic'],
 ['tax', 'roth', 'ira', 'retire', 'live', 'estate', 'goal']]

#### Non-negative matrix factorization (NMF), TfidfVectorizer

In [15]:
topics_nmf_tfidf = topic_model(X_tfidf, NMF, 12, 7)[0]
topics_nmf_tfidf

[['index', 'life', 'cheap', 'warren', 'bond', 'manager', 'answer'],
 ['constant', 'formula', 'equal', 'divide', 'present', 'minus', 'calculate'],
 ['moat', 'mode', 'competitive', 'economic', 'competitor', 'brand', 'castle'],
 ['etf', 'holding', 'vanguard', 'index', 'expense', 'etfs', 'sector'],
 ['ebitda', 'enterprise', 'multiple', 'irr', 'da', 'forecast', 'decade'],
 ['apple', 'cent', 'facebook', 'microsoft', 'amazon', 'iphone', 'google'],
 ['foreign', 'applause', 'raider', 'backbone', 'nifty', 'twice', 'prior'],
 ['intrinsic',
  'discount',
  'method',
  'calculate',
  'graham',
  'safety',
  'estimate'],
 ['technical', 'trader', 'chart', 'trend', 'volume', 'shortterm', 'statement'],
 ['tesla', 'car', 'vehicle', 'elon', 'battery', 'neo', 'electric'],
 ['pe', 'eps', 'divide', 'multiple', 'peg', 'amazon', 'metric'],
 ['bank', 'loan', 'sector', 'rupee', 'statement', 'deposit', 'liability']]

#### Latent Dirichlet Allocation (LDA), CountVectorizer

In [16]:
topics_lda_cv = topic_model(X_cv, LatentDirichletAllocation, 6, 7)[0]
topics_lda_cv

[['warren', 'graham', 'life', 'stuff', 'berkshire', 'write', 'wrong'],
 ['index', 'bond', 'inflation', 'sp', 'factor', 'mutual', 'cheap'],
 ['moat', 'bank', 'customer', 'competitive', 'brand', 'economic', 'mode'],
 ['tax', 'goal', 'estate', 'etf', 'holding', 'live', 'individual'],
 ['discount', 'divide', 'calculate', 'pe', 'equal', 'formula', 'multiple'],
 ['etf', 'tesla', 'apple', 'option', 'amazon', 'cent', 'list']]

### Topic modeling - Adjectives

In [17]:
df_videos_cleaned_v9_adj = df_videos_cleaned_v9.copy()

In [18]:
df_videos_cleaned_adj = corpus_of_adjectives(df_videos_cleaned_v9_adj)

In [19]:
X_cv_adj = document_term_matrix(df_videos_cleaned_adj, CountVectorizer)[0]

In [20]:
X_tfidf_adj = document_term_matrix(df_videos_cleaned_adj, TfidfVectorizer)[0]

#### Non-negative matrix factorization (NMF), CountVectorizer

In [21]:
topics_nmf_cv_adj = topic_model(X_cv_adj, NMF, 6, 6)[0]
topics_nmf_cv_adj

[['brownforman', 'bahamas', 'beget', 'allow', 'cardinal', 'airline'],
 ['brownfield', 'break', 'burt', 'airline', 'cannula', 'caltech'],
 ['amalgam', 'candor', 'brew', 'break', 'acl', 'allianz'],
 ['boy', 'aj', 'break', 'brazilian', 'bulk', 'bookkeeper'],
 ['article', 'allianz', 'attain', 'candor', 'cannula', 'behaviorally'],
 ['beth', 'candor', 'assured', 'alumnus', 'alexis', 'commissionfree']]

#### Non-negative matrix factorization (NMF), TfidfVectorizer

In [22]:
topics_nmf_tfidf_adj = topic_model(X_tfidf_adj, NMF, 6, 6)[0]
topics_nmf_tfidf_adj

[['beget', 'brownforman', 'brownfield', 'bracelet', 'aj', 'bus'],
 ['amalgam', 'candor', 'article', 'brownfield', 'allianz', 'canberra'],
 ['beth', 'assured', 'commissionfree', 'candor', 'alumnus', 'compose'],
 ['allow', 'archaic', 'cocaine', 'contributes', 'beraldo', 'arab'],
 ['coil', 'adore', 'ageappropriate', 'burt', 'aum', 'airline'],
 ['break', 'boy', 'brazilian', 'bulk', 'caltech', 'acl']]

#### Latent Dirichlet Allocation (LDA), CountVectorizer

In [23]:
topics_lda_cv_adj = topic_model(X_cv_adj, LatentDirichletAllocation, 6, 6)[0]
topics_lda_cv_adj

[['coil', 'convince', 'bsf', 'chrome', 'bahamas', 'clue'],
 ['allow', 'archaic', 'cocaine', 'anz', 'bookkeeper', 'axiom'],
 ['article', 'candor', 'amalgam', 'brownfield', 'beth', 'allianz'],
 ['aj', 'boy', 'beth', 'break', 'brazilian', 'assured'],
 ['brownfield', 'break', 'caltech', 'burt', 'brazilian', 'backup'],
 ['beget', 'brownforman', 'bracelet', 'bulletin', 'bus', 'blatantly']]

### Assigning topics and coefficients to videos

In [24]:
df_videos_cleaned_v10 = topic_assignment(df_videos_cleaned_v9)

In [25]:
df_videos_cleaned_v10['Topic'].value_counts()

Value Investing                       745
Dividend Investing                    212
Technology Stocks                     173
Economic Moats                        129
Passive Investing                     121
Valuation                             113
Fundamental vs. Technical Analysis    112
Electric Vehicle Stocks                84
Valuation (Case Studies)               64
General                                25
Name: Topic, dtype: int64

### Reassigning topics for videos under 'Value Investing'

In [26]:
df_videos_cleaned_v10_value = df_videos_cleaned_v10[df_videos_cleaned_v10['Topic']=='Value Investing'].sort_values('Topic Coefficient')
df_videos_cleaned_v10_other = df_videos_cleaned_v10[df_videos_cleaned_v10['Topic']!='Value Investing']

In [27]:
df_videos_cleaned_v10_value.loc[1355, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[94, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[86, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[187, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1458, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1408, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[669, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1632, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1557, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1207, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[348, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1556, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[98, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1641, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1328, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[152, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1363, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[242, 'Topic'] = 'Economic Moats'
df_videos_cleaned_v10_value.loc[752, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1204, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1333, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1537, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1693, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[374, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1360, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1189, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[448, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[931, 'Topic'] = 'Passive Investing'
df_videos_cleaned_v10_value.loc[1253, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1340, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[834, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1496, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1704, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1414, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[709, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[575, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1358, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1361, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1771, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1523, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[300, 'Topic'] = 'Economic Moats'
df_videos_cleaned_v10_value.loc[948, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1137, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1498, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1143, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1133, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[298, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[703, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[699, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1590, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1224, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1173, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1158, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[278, 'Topic'] = 'Economic Moats'
df_videos_cleaned_v10_value.loc[33, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1402, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1263, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1171, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1735, 'Topic'] = 'General'	
df_videos_cleaned_v10_value.loc[1591, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1260, 'Topic'] = 'General'	
df_videos_cleaned_v10_value.loc[1293, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1149, 'Topic'] = 'General'	
df_videos_cleaned_v10_value.loc[1261, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[113, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1443, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[743, 'Topic'] = 'General'		
df_videos_cleaned_v10_value.loc[1255, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1487, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[246, 'Topic'] = 'Economic Moats'
df_videos_cleaned_v10_value.loc[1284, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1296, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1225, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1001, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1509, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1609, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1140, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1645, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[538, 'Topic'] = 'Technology Stocks'
df_videos_cleaned_v10_value.loc[1649, 'Topic'] = 'Growth Investing vs. Value Investing'	
df_videos_cleaned_v10_value.loc[1411, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1447, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[859, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1736, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[760, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1492, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1091, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1708, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1516, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1089, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1407, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1130, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[161, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1103, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1262, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[386, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[454, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[245, 'Topic'] = 'Economic Moats'
df_videos_cleaned_v10_value.loc[1629, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[510, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1714, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[119, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[188, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[433, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1305, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1203, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1362, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[620, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1486, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1503, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1327, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1353, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1269, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[104, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[1178, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1064, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[482, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[829, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1671, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1309, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[38, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1734, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1304, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1250, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1628, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[151, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[792, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1676, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1664, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1697, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[860, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1742, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1182, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[180, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1570, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1315, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1663, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1597, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1344, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1698, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[655, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1462, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1251, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[637, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1110, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1210, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[651, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[594, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[144, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1345, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[342, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1386, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1655, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1283, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1167, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1380, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1438, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[721, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[624, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[194, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1400, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[78, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1272, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1433, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1222, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1702, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1519, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1493, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[840, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1245, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1773, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[677, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1478, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1555, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1578, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[16, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1376, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1067, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1385, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1403, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1618, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1256, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[500, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1330, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[92, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[365, 'Topic'] = 'Economic Moats'
df_videos_cleaned_v10_value.loc[128, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1357, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1048, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1220, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1405, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1217, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1470, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[782, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1104, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[937, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1518, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1653, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[711, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[823, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1482, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1636, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[563, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1359, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[405, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1352, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1368, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1025, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[642, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1175, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1019, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[621, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1148, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[35, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1145, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1725, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1322, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1320, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1543, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[558, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1206, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1192, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[11, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[1208, 'Topic'] = 'Passive Investing'
df_videos_cleaned_v10_value.loc[670, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1567, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1152, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[632, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[855, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1332, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1168, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[557, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[578, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[808, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1592, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1476, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[63, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[388, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1141, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[571, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1098, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[911, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[399, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1688, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[124, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[3, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1457, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1187, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1489, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1370, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[22, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1471, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[411, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1242, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1233, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1216, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[857, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[746, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1546, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1169, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1768, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1406, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1230, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1096, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[997, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1097, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1679, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[652, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[679, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[657, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1114, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1228, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1277, 'Topic'] = 'Passive Investing'
df_videos_cleaned_v10_value.loc[1279, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[706, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1379, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[896, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1435, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1445, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1659, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1246, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1425, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1755, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[959, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1350, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1086, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1701, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1620, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1313, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1568, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1505, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[906, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1005, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[19, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1711, 'Topic'] = 'Technology Stocks'
df_videos_cleaned_v10_value.loc[1365, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1087, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[441, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[813, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[282, 'Topic'] = 'Economic Moats'
df_videos_cleaned_v10_value.loc[375, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[785, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1440, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[64, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1517, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[905, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1095, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[52, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1683, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1637, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[47, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1307, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1477, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[826, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[781, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[688, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1594, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[322, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1311, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[55, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[381, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[50, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1491, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1643, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[368, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[134, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1274, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[680, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[501, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[117, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1598, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[932, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[536, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1069, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1212, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[415, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[82, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[497, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1180, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1510, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[56, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1371, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[466, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[992, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[609, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1113, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[129, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1613, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1488, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1286, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1538, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1575, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1348, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1640, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[702, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1155, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1200, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[425, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[589, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1661, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[14, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1642, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[72, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[184, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1524, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1602, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1115, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[147, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[684, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[89, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[125, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1213, 'Topic'] = 'Passive Investing'
df_videos_cleaned_v10_value.loc[1300, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[613, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[449, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[692, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1194, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[382, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1600, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[779, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1231, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1685, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[1127, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[549, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1119, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[641, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1614, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[8, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[211, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[193, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1384, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1772, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[439, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[4, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[315, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1044, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[385, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[66, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1341, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1480, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[451, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[980, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1270, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1314, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1448, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1101, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1190, 'Topic'] = 'Passive Investing'
df_videos_cleaned_v10_value.loc[1185, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[630, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1616, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1239, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1573, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[747, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[192, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1254, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1677, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[734, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1566, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[481, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[648, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1757, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[745, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[51, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[177, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1081, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1499, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1090, 'Topic'] = 'Long-term Investing'
df_videos_cleaned_v10_value.loc[1021, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[537, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[159, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1770, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1013, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[708, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1297, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[9, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[12, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[250, 'Topic'] = 'Economic Moats'
df_videos_cleaned_v10_value.loc[1581, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[672, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[650, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[685, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[443, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[99, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[800, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[1132, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[539, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[567, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[534, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1290, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[635, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1068, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[712, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[850, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[663, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[640, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[173, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[102, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[141, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[298, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[551, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[521, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1436, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[37, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[395, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[700, 'Topic'] = 'Technology Stocks'
df_videos_cleaned_v10_value.loc[6, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[115, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[547, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[852, 'Topic'] = 'Growth Investing vs. Value Investing'
df_videos_cleaned_v10_value.loc[45, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[522, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1337, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[714, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[499, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[148, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[667, 'Topic'] = 'Technology Stocks'
df_videos_cleaned_v10_value.loc[775, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[936, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[530, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[5, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[603, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[491, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[114, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[647, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[664, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[156, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1586, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[393, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[638, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[724, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[470, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[627, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[675, 'Topic'] = 'Technology Stocks'
df_videos_cleaned_v10_value.loc[576, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[535, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[573, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[585, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[773, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1681, 'Topic'] = 'Dividend Investing'
df_videos_cleaned_v10_value.loc[529, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[377, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[728, 'Topic'] = 'Technology Stocks'
df_videos_cleaned_v10_value.loc[682, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[516, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[378, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[717, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1501, 'Topic'] = 'Technology Stocks'
df_videos_cleaned_v10_value.loc[0, 'Topic'] = 'Fundamental vs. Technical Analysis'
df_videos_cleaned_v10_value.loc[555, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[169, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[541, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[68, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[540, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[626, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[542, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1034, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[583, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[1366, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[868, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[750, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[469, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[895, 'Topic'] = 'General'
df_videos_cleaned_v10_value.loc[1223, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[409, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[383, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[176, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[548, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[406, 'Topic'] = 'Valuation'
df_videos_cleaned_v10_value.loc[846, 'Topic'] = 'Valuation'

In [28]:
df_videos_cleaned_v10 = pd.concat([df_videos_cleaned_v10_value, df_videos_cleaned_v10_other])

In [29]:
# df_videos_cleaned_v10 = df_videos_cleaned_v10.replace('Growth Investing vs. Value Investing', 'Growth Investing Vs. Value Investing')
# df_videos_cleaned_v10 = df_videos_cleaned_v10.replace('Fundamental vs. Technical Analysis', 'Fundamental Vs. Technical Analysis')
df_videos_cleaned_v10 = df_videos_cleaned_v10.drop([1133])

In [30]:
df_videos_cleaned_v10['Topic'].value_counts()

General                                 264
Dividend Investing                      257
Value Investing                         218
Valuation                               195
Technology Stocks                       180
Economic Moats                          137
Passive Investing                       126
Fundamental vs. Technical Analysis      123
Electric Vehicle Stocks                  83
Long-term Investing                      71
Valuation (Case Studies)                 64
Growth Investing vs. Value Investing     59
Name: Topic, dtype: int64

### Pickle the modified dataframe 

In [31]:
pickle_df('df_videos_cleaned_v10')